# # Fetch New Data since select Date to Parquet files

In [12]:
from arctic import Arctic,CHUNK_STORE
import ccxt
from datetime import datetime as dt
import os
from pathlib import Path    
import pandas as pd
import sys
import time
from tqdm import tqdm


    
def first_install(db):
    try:
        db.initialize_library('eod')
    except:
        print('Already Created')
        
    collection = db['eod']
    return collection

# def __exit__(self, exc_type, exc_value, traceback):
#     self.db.delete_library('eod')

def get_storage_size(collection):
    storage_size  = round(collection.stats()['totals']['size']/(1024*1024),3)
    print(f'Total Size {storage_size} MBs ')
    return  storage_size

def write_one(collection,ticker,df):
    collection.write(ticker, df)

def list_tickers(collection):
    return collection.list_symbols()

def read_one(collection, ticker):
    return collection.read(ticker).data

def delete_one(collection, ticker):
    collection.delete(ticker)

def append_one( collection,ticker, df):
    collection.append(ticker, df, upsert=False)
 
ex =ccxt.ftx()
store = Arctic('localhost')   

In [31]:
# help(read_one(collection,'SOL'))

In [26]:
collection = store['eod']

In [1]:
# -*- coding: utf-8 -*-
###### FETCH DATA To Parquet files ############################
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# common constants

msec = 1000
minute = 60 * msec

all_data = []
ftx = ccxt.ftx()

# symbols_list = ['BTC/USD','ETH/USD','FTT/USD','BNB/USD','SOL/USD']
symbols_list = ['ATLAS/USD','AAVE/USD','AXS/USD']


# from_datetime = '2020-01-01 00:00:00'
# from_timestamp = ftx.parse8601(from_datetime)

now = ftx.milliseconds()
time_str  =  dt.now().strftime('%Y-%m-%d %H:%M:%S')

# symbols=  'SOL/USD'
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------

print(f'Start Time [{time_str}]')
from_datetime = '2020-01-01 00:00:00'

for i in symbols_list:
    name = i.split('/')[0]
    from_timestamp = ftx.parse8601(from_datetime)
    
    start_str  = dt.fromtimestamp(from_timestamp/1000).strftime('%Y%m%d_%H%M')
    print('Check Start Since  ',start_str ,'  ', name)
    check_day_ts =        ftx.fetch_ohlcv(i, '1d')
    first_day_ts  =      check_day_ts[0][0] # first ts on row
    print('Check First Day on Exchange',i,ftx.iso8601(first_day_ts))
    
    # Force filter date to First list on exchange
    if  from_timestamp <= first_day_ts:
        from_timestamp = first_day_ts
    print(dt.fromtimestamp(from_timestamp/1000).strftime('%Y%m%d_%H%M'))
    print('#'*50)
    # Loop untils nows 

    while from_timestamp < now:


        ohlcvs = ftx.fetch_ohlcv(i, '1m', from_timestamp)
        print('Fetching candles starting from', ftx.iso8601(from_timestamp))


        # don't hit the rateLimit or you will be banned
        time.sleep(ftx.rateLimit / msec)
        if ohlcvs !=[]:
            # ftx returns 1000 candles for 1m timeframe at once
            from_timestamp += len(ohlcvs) * minute                                    
            # Append Data on empty list
            all_data.append(ohlcvs)
        else :
            from_timestamp += 1000 *minute
            
            
    time_str  =  dt.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'Ending Time Time [{time_str}]')
    end_str = dt.fromtimestamp(now/1000).strftime('%Y%m%d_%H%M')

    df=  pd.DataFrame(all_data[0])
    cols=['ts','o','h','l','c','v']
    
    ##### Processing Data to Dataframe
    for i in tqdm(range(len(all_data))): ###
            if i+1 <= len(all_data)-1:
                df= df.append(all_data[i+1])
    df.columns = cols
    df['ts'] =  df['ts'].apply( ftx.iso8601)
    df.to_parquet(f'{name}_{start_str}_{end_str}.parquet')


NameError: name 'ccxt' is not defined

In [3]:
#check
check_day_ts =        ftx.fetch_ohlcv('BTC/USD', '1d', )
first_day_ts  =      check_day_ts[0][0] # first ts on row
print('Check First Day on Exchange',i,ftx.iso8601(first_day_ts))

Check First Day on Exchange ATLAS/USD 2019-07-21T00:00:00.000Z


In [4]:
#check

ftx.fetch_ohlcv(i, '1m', 1630540800000)
# ftx.parse8601(1630540800000)
# ftx.iso8601(1630540800000)

[[1630587540000, 0.001399875, 0.001399875, 0.001399875, 0.001399875, 0.0],
 [1630587600000, 0.001399875, 0.17, 0.001399875, 0.13, 2893505.74025],
 [1630587660000, 0.13, 0.141959, 0.088, 0.09, 3238535.70075],
 [1630587720000, 0.09, 0.094, 0.069, 0.091102, 2261527.68675],
 [1630587780000, 0.091102, 0.11, 0.09, 0.094, 1884652.47225],
 [1630587840000, 0.094, 0.118, 0.0926125, 0.095, 1649968.97375],
 [1630587900000, 0.095, 0.1, 0.07149925, 0.076725, 924286.17725],
 [1630587960000, 0.076725, 0.085, 0.07018125, 0.07955475, 749624.02625],
 [1630588020000, 0.07955475, 0.0799115, 0.0733, 0.0769825, 427635.202],
 [1630588080000, 0.0735, 0.0769825, 0.07, 0.07, 617152.813],
 [1630588140000, 0.07, 0.0790805, 0.07, 0.07700025, 674879.21925],
 [1630588200000, 0.07700025, 0.07931225, 0.071, 0.07499025, 414564.988],
 [1630588260000, 0.07499025, 0.085218, 0.07220025, 0.085218, 549156.53575],
 [1630588320000, 0.085218, 0.0938, 0.085, 0.0889015, 349291.728],
 [1630588380000, 0.0889015, 0.089, 0.08010025, 0

# # Insert all parquet files to Artic-db

In [2]:

### Read all parquet files and insert to Arctic - Mongodb
data_dir = Path('C:\\Users\\User\\Desktop\\sahing')
for i, parquet_path in enumerate(data_dir.glob('*.parquet')):
#     print('Name :',parquet_path)
    df = pd.read_parquet(parquet_path)
    ticker   = parquet_path._str.split('\\')[-1].split('_')[0]
#     print(ticker)
    size = (df.__sizeof__(),': Bytes')
    collection = first_install(store)
    write_one(collection,ticker,df)
    #check Size of arctic data
    data =read_one(collection,ticker)
    size_arctic  = data.size

    print(f'Write {ticker}  {size } to  {size_arctic} Arctic Mongodb')


Library created, but couldn't enable sharding: no such command: 'enablesharding', full error: {'ok': 0.0, 'errmsg': "no such command: 'enablesharding'", 'code': 59, 'codeName': 'CommandNotFound'}. This is OK if you're not 'admin'


Write BNB  (120325395, ': Bytes') to  5596530 Arctic Mongodb


Library created, but couldn't enable sharding: no such command: 'enablesharding', full error: {'ok': 0.0, 'errmsg': "no such command: 'enablesharding'", 'code': 59, 'codeName': 'CommandNotFound'}. This is OK if you're not 'admin'


Write BTC  (120325395, ': Bytes') to  5596530 Arctic Mongodb


Library created, but couldn't enable sharding: no such command: 'enablesharding', full error: {'ok': 0.0, 'errmsg': "no such command: 'enablesharding'", 'code': 59, 'codeName': 'CommandNotFound'}. This is OK if you're not 'admin'


Write ETH  (120325395, ': Bytes') to  5596530 Arctic Mongodb


Library created, but couldn't enable sharding: no such command: 'enablesharding', full error: {'ok': 0.0, 'errmsg': "no such command: 'enablesharding'", 'code': 59, 'codeName': 'CommandNotFound'}. This is OK if you're not 'admin'


Write FTT  (120325395, ': Bytes') to  5596530 Arctic Mongodb


Library created, but couldn't enable sharding: no such command: 'enablesharding', full error: {'ok': 0.0, 'errmsg': "no such command: 'enablesharding'", 'code': 59, 'codeName': 'CommandNotFound'}. This is OK if you're not 'admin'


Write SOL  (120325395, ': Bytes') to  5596530 Arctic Mongodb


In [198]:
for i,path in enumerate(data_dir.glob('*.parquet')):
    print(path)
#     dir(data_dir)

C:\Users\User\Desktop\sahing\AAVE_20200101_0700_20211014_0132.parquet
C:\Users\User\Desktop\sahing\ATLAS_20200101_0700_20211014_0132.parquet
C:\Users\User\Desktop\sahing\AXS_20200101_0700_20211014_0132.parquet
C:\Users\User\Desktop\sahing\BNB_20211010_0055_20211010_0048.parquet
C:\Users\User\Desktop\sahing\BTC_20200101_0700_20211010_0048.parquet
C:\Users\User\Desktop\sahing\ETH_20211010_0055_20211010_0048.parquet
C:\Users\User\Desktop\sahing\FTT_20211010_0055_20211010_0048.parquet
C:\Users\User\Desktop\sahing\SOL_20211010_0055_20211010_0048.parquet


In [181]:
# check size 
collection = store['eod']
get_storage_size(collection)

Total Size 135.132 MBs 


135.132

In [182]:
last_dt_ts
print(len(str(last_dt_ts)),last_dt_ts)

13 1634148000371


In [183]:
now = ex.milliseconds()

print(len(str(now)),now)

13 1634150858063


In [184]:
### Check Ticker list name
list_tick =list_tickers(collection)
symbols_list = [symbol+'/USD' for symbol in  list_tick]
print('DB Ticker Name :' , list_tick)
print('Symbols Name : ',symbols_list)

DB Ticker Name : ['BNB', 'BTC', 'ETH', 'FTT', 'SOL']
Symbols Name :  ['BNB/USD', 'BTC/USD', 'ETH/USD', 'FTT/USD', 'SOL/USD']


In [185]:
# Read all Data in arctic db and append on dictionary

all_data   = []
all_data_dict  = {}
for i  in list_tickers(collection) :
    data =  collection.read(i).data
    data.set_index('ts',inplace=True)
    aa = data[['c']].reset_index().to_dict(orient='list')
    data_dict   =  {'symbol':i,'timestamp':aa['ts']  ,  'price':aa['c']}
    all_data.append(data_dict)
    all_data_dict[i] =  {'timestamp':aa['ts']  ,  'price':aa['c']}
print('Show All Symbols ',all_data_dict.keys())

KeyboardInterrupt: 

In [63]:
# Check Last timestamp from DB
last_dt_str  = read_one(collection,ticker)['ts'].iloc[-1]
last_dt_ts = ex.parse8601(last_dt_str)
print(ticker,last_dt_ts ,' //' , last_dt_str)

SOL 1634147820000  // 2021-10-13T17:57:00.000Z


In [188]:
# Edit Test 
list_tick = [list_tick[-1]]
symbols_list = [symbol+'/USD' for symbol in  list_tick]
print('DB Ticker Name :' , list_tick)
print('Symbols Name : ',symbols_list)
print(ex.rateLimit /1000)

DB Ticker Name : ['SOL']
Symbols Name :  ['SOL/USD']
0.1


# Fill Data to Arctic and  save to paqruet


In [65]:
import time 
#### Fetch New Data and append on Artic DB



msec = 1000
minute = 60 * msec

now = ex.milliseconds()
all_data = {}
#
for i in range(len(list_tick)):
    name = list_tick[i]
    symbols_name =symbols_list[i]
    df= read_one(collection,name)
    count_df =  len(df)
    last_dt_str=  df.tail(1)['ts'].values[0]
    last_dt_ts =  ex.parse8601(last_dt_str)
    print(f'{list_tick[i]} -__- {symbols_list[i]}')

    # 
    while last_dt_ts < now:
        mark_ts = last_dt_ts
        print(name,'FETCH FILL',last_dt_ts < now)
    
        fill_data   = ex.fetch_ohlcv(symbols_name, '1m', since= last_dt_ts ,limit=None)
        
#         print('Rate Limit     ',  ex.rateLimit / msec)

#         time.sleep(ftx.rateLimit / msec) # 0.1
        time.sleep(0.5)
        # Append Data on empty list
        all_data[name] =fill_data

        if len(fill_data)  >=  5:

            #### 1 ways  
            update_df  = pd.DataFrame(all_data[name])
            cols=['ts','o','h','l','c','v']
            update_df.columns =    cols  
            update_df['ts'] =  update_df['ts'].apply(ex.iso8601)
            #skip first row
            update_df = update_df[1:]
            append_one(collection,list_tick[i],update_df)
            last_dt_str=  update_df.tail(1)['ts'].values[0]
            last_dt_ts =  ex.parse8601(last_dt_str)
            print(f'Success Update Data ')
            print('#'*50)
        else :
            print('Small Data to append  OR Finshed ')
            print(f'{last_dt_ts}')
            print('#'*50)
            last_dt_ts = now
            
            
            
        print('OLD  TS ', mark_ts , ' Last TS ',last_dt_ts )

    ### 2 ways    
#     update_df  = pd.DataFrame(fill_data)
#     cols=['ts','o','h','l','c','v']
#     update_df.columns =    cols  
#     update_df['ts'] =  update_df['ts'].apply(ex.iso8601)
#     end_update  = pd.concat([df,update_df])
#     end_update =end_update.drop_duplicates()    
#     up_date_dt_str =  end_update.tail(1)['ts'].values[0]
#     up_date_dt_ts =ex.parse8601(up_date_dt_str)
#     count_update  = len(end_update)
#     print(up_date_dt_ts,up_date_dt_str,count_update)
#     append_one(collection,list_tick[i],end_update)

    time.sleep(1)
    # append data to arctic db 
    get_storage_size(collection)
    print('#'*50)


BNB -__- BNB/USD
BNB FETCH FILL True
Small Data to append  OR Finshed 
1634147940000
##################################################
OLD  TS  1634147940000  Last TS  1634148000371
Total Size 135.132 MBs 
##################################################
BTC -__- BTC/USD
BTC FETCH FILL True
Small Data to append  OR Finshed 
1634147940000
##################################################
OLD  TS  1634147940000  Last TS  1634148000371
Total Size 135.132 MBs 
##################################################
ETH -__- ETH/USD
ETH FETCH FILL True
Small Data to append  OR Finshed 
1634147940000
##################################################
OLD  TS  1634147940000  Last TS  1634148000371
Total Size 135.132 MBs 
##################################################
FTT -__- FTT/USD
FTT FETCH FILL True
Small Data to append  OR Finshed 
1634147940000
##################################################
OLD  TS  1634147940000  Last TS  1634148000371
Total Size 135.132 MBs 
###################

In [190]:
# Loop all get perpetual symbols 
a= []
for i in ex.load_markets():
#     print(i)
    if '-PERP' in i:
        a.append(i)
        
        
        

In [116]:
(all_market[i]['info'])

{'name': 'ZECBULL/USD',
 'enabled': True,
 'postOnly': False,
 'priceIncrement': 0.0001,
 'sizeIncrement': 0.1,
 'minProvideSize': 0.1,
 'last': 0.3273,
 'bid': 0.3244,
 'ask': 0.3279,
 'price': 0.3273,
 'type': 'spot',
 'baseCurrency': 'ZECBULL',
 'quoteCurrency': 'USD',
 'underlying': None,
 'restricted': False,
 'highLeverageFeeExempt': True,
 'change1h': 0.02537593984962406,
 'change24h': 0.03216650898770104,
 'changeBod': 0.043021032504780114,
 'quoteVolume24h': 15900.03438,
 'volumeUsd24h': 15900.03438}

In [9]:
# all spot Name 
all_market = ex.load_markets()
all_list = []
for i in all_market:
    if 'USD' in i:
        if not 'USDT' in i:
            volume_day  = float(all_market[i]['info']['volumeUsd24h'])
#             print(type(volume_day),volume_day)
            if volume_day>= 1000000 :#
                sym_dict   = {'symbols':i,'volume':int(round(volume_day,2))}
                all_list.append(sym_dict)
sort_vol  = pd.DataFrame(all_list).sort_values(by='volume',ascending=False)
sort_vol.head(20)

#     if 'USD' in i:
#         if not 'USDT' in i:
#             print(i)

,symbols,volume
15,BTC/USD,736647761
28,ETH/USD,714084070
57,SOL/USD,300214417
24,DOGE/USD,74813110
53,SAND/USD,68835199
31,FTT/USD,57751861
13,BNB/USD,55276534
71,XRP/USD,47767833
41,LTC/USD,37736659
5,ATLAS/USD,30081804


In [10]:
read_one(collection,'BNB')

NameError: name 'collection' is not defined

In [ ]:
append_one(collection,'BNB',n_df)